In [1]:
import torch
import numpy as np
from torch.autograd import Variable
import json

In [2]:
def json2edgelist(x):
    edges=[]
    for i in x.keys():
        for j in x[i].keys():
            edges.append([i,j,x[i][j]])
    edges2=np.array(edges)
    m=edges2[:,2].astype(int)
    m=m/np.sum(m)
    return(edges2[:,:2],m)
            

In [3]:
d=json.loads(open("./pr/data/Gt.json","rb").read())
edges,m=json2edgelist(d)

In [4]:
nodes=np.unique(edges.flatten())
node2i={wq:qw for qw,wq in enumerate(nodes)}

In [5]:
# Creating the graph
if torch.cuda.is_available():
    print("Yaay!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    cuda1 = torch.device('cuda:0')
lambd_t = Variable(torch.rand(len(nodes)).cuda(cuda1).float() , requires_grad = True)
muv_=Variable(torch.tensor(m).cuda(cuda1).float())

def exp3(lambd_t):
    lambd_t_i= torch.index_select(lambd_t, 0, Variable(torch.tensor([node2i[i[0]] for i in edges]).cuda(cuda1)))
    lambd_t_j= torch.index_select(lambd_t, 0, Variable(torch.tensor([node2i[i[1]] for i in edges]).cuda()))
    s=torch.sum(muv_*torch.exp(lambd_t_i-lambd_t_j)) #minimize \sum_{u,v}{p_{u,v}} to maximize (1-\sum_{u,v}{p_{u,v}})
    return(s)


















'''








def calp(lambd_t):
    lambd_t_i= torch.index_select(lambd_t, 0, Variable(torch.tensor([node2i[i[0]] for i in edges]).cuda(cuda1))).data
    lambd_t_j= torch.index_select(lambd_t, 0, Variable(torch.tensor([node2i[i[1]] for i in edges]).cuda())).data
    ps=(muv_/(1.0+lambd_t_j-lambd_t_i))
    print(torch.sum(ps).item())
    ps=ps/ps.max().item()
    return(ps)
def get_bins(ll):
    return([torch.sum((ll.data>(ff/10.0)) & (ll.data<=(0.1+(ff/10.0)))).item() for ff in range(10)])
def get_bins2(ll):
    print("divergence",ll.std()/ll.mean())
    return([torch.sum((ll.data>(ff/100.0)) & (ll.data<=(0.01+(ff/100.0)))).item() for ff in range(10)]+[torch.sum((ll.data>0.1)).item()])
''''''
def maximize_exp(lambd1_t,lambd_t):
    lambd_t_i= torch.index_select(lambd_t, 0, Variable(torch.tensor([node2i[i[0]] for i in edges])))
    lambd_t_j= torch.index_select(lambd_t, 0, Variable(torch.tensor([node2i[i[1]] for i in edges])))
    puv_t=muv_/(lambd1_t+lambd_t_j-lambd_t_i)
    s=torch.sum(-muv_*torch.log(puv_t**2)/2.0/np.log(2))+(torch.sum(puv_t)-1.0)*lambd1_t+torch.sum((lambd_t_j-lambd_t_i)*puv_t)
    return(s,torch.sum(puv_t),torch.sum(torch.abs(puv_t)))

def exp2(lambd1_t,lambd_t):
    lambd_t_i= torch.index_select(lambd_t, 0, Variable(torch.tensor([node2i[i[0]] for i in edges])))
    lambd_t_j= torch.index_select(lambd_t, 0, Variable(torch.tensor([node2i[i[1]] for i in edges])))
    puv_t=muv_/(lambd1_t+lambd_t_j-lambd_t_i)
    s=torch.sum(-muv_*torch.log(puv_t**2)/2.0/np.log(2))+(torch.sum(puv_t)-1.0).pow(2)
    return(torch.sum((lambd_t_i-lambd_t_j).pow(2)))'''

Yaay!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


'\n\n\n\n\n\n\n\n\ndef calp(lambd_t):\n    lambd_t_i= torch.index_select(lambd_t, 0, Variable(torch.tensor([node2i[i[0]] for i in edges]).cuda(cuda1))).data\n    lambd_t_j= torch.index_select(lambd_t, 0, Variable(torch.tensor([node2i[i[1]] for i in edges]).cuda())).data\n    ps=(muv_/(1.0+lambd_t_j-lambd_t_i))\n    print(torch.sum(ps).item())\n    ps=ps/ps.max().item()\n    return(ps)\ndef get_bins(ll):\n    return([torch.sum((ll.data>(ff/10.0)) & (ll.data<=(0.1+(ff/10.0)))).item() for ff in range(10)])\ndef get_bins2(ll):\n    print("divergence",ll.std()/ll.mean())\n    return([torch.sum((ll.data>(ff/100.0)) & (ll.data<=(0.01+(ff/100.0)))).item() for ff in range(10)]+[torch.sum((ll.data>0.1)).item()])\n\ndef maximize_exp(lambd1_t,lambd_t):\n    lambd_t_i= torch.index_select(lambd_t, 0, Variable(torch.tensor([node2i[i[0]] for i in edges])))\n    lambd_t_j= torch.index_select(lambd_t, 0, Variable(torch.tensor([node2i[i[1]] for i in edges])))\n    puv_t=muv_/(lambd1_t+lambd_t_j-lambd_t_i

In [6]:
opt = torch.optim.Adam(params=[lambd_t])

In [ ]:
torch.sum(muv_)

tensor(1., device='cuda:0')

In [ ]:
for i in range(900):
    opt.zero_grad()
    loss2=exp3(lambd_t)
    loss2.backward(retain_graph=True)
    opt.step()
    print("\n",loss2.tolist(),lambd_t.grad.min().item(),lambd_t.grad.max().item(),lambd_t.min().item(),lambd_t.max().item(),lambd_t.grad.median().item())


 1.2193658351898193 -0.20474138855934143 0.002036724705249071 -0.0009398761321790516 1.000978708267212 3.634712584243971e-07

 1.2170543670654297 -0.20434923470020294 0.002032662509009242 -0.0019317509140819311 1.0019785165786743 3.628976799063821e-07

 1.214730978012085 -0.20395527780056 0.002028608927503228 -0.002925005042925477 1.0029782056808472 3.6231747913006984e-07

 1.2124042510986328 -0.2035582810640335 0.0020245625637471676 -0.003919013775885105 1.0039777755737305 3.617269328515249e-07

 1.2100770473480225 -0.20316462218761444 0.0020205241162329912 -0.0049134669825434685 1.0049772262573242 3.6110196788285975e-07

 1.2077511548995972 -0.20276618003845215 0.0020164947491139174 -0.005908174905925989 1.0059764385223389 3.6049479490429803e-07

 1.2054274082183838 -0.20237094163894653 0.0020124749280512333 -0.006903006229549646 1.0069754123687744 3.598717057684553e-07

 1.20310640335083 -0.20197655260562897 0.0020084641873836517 -0.007897862233221531 1.0079741477966309 3.592403174

In [ ]:
#This will always go down, the constraints are not met.. not the solution we are looking for.. infimum problem? not in the domain?